In [1]:
import pandas as pd

In [ ]:
file_path = r"../data/chronos/chronos_AAPL_data.csv"
data = pd.read_csv(file_path)

# It Changes to object after saving
data['timestamp'] = pd.to_datetime(data['timestamp'])

data

,item_id,timestamp,target,Open,High,Low,Volume
0,AAPL,2022-03-21 09:30:00,0.287312,0.293826,0.300777,0.288082,0.320497
1,AAPL,2022-03-21 10:30:00,0.308235,0.287211,0.304994,0.289602,0.265453
2,AAPL,2022-03-21 11:30:00,0.305077,0.308134,0.306400,0.305547,0.174637
3,AAPL,2022-03-21 12:30:00,0.290735,0.305050,0.301184,0.289158,0.177918
4,AAPL,2022-03-21 13:30:00,0.298127,0.290705,0.294858,0.291086,0.148927
...,...,...,...,...,...,...,...
5078,AAPL,2025-02-13 15:30:00,0.869258,0.869372,0.864440,0.866508,0.074582
5079,AAPL,2025-02-14 09:30:00,0.890516,0.865841,0.888716,0.866434,0.197734
5080,AAPL,2025-02-14 10:30:00,0.887283,0.890443,0.883759,0.882453,0.086259
5081,AAPL,2025-02-14 11:30:00,0.886318,0.887173,0.884869,0.886384,0.073202


In [39]:
from autogluon.timeseries import TimeSeriesDataFrame

ts_data = TimeSeriesDataFrame(data)
ts_data

target      Open      High       Low    Volume
item_id timestamp                                                            
AAPL    2022-03-21 09:30:00  0.287312  0.293826  0.300777  0.288082  0.320497
        2022-03-21 10:30:00  0.308235  0.287211  0.304994  0.289602  0.265453
        2022-03-21 11:30:00  0.305077  0.308134  0.306400  0.305547  0.174637
        2022-03-21 12:30:00  0.290735  0.305050  0.301184  0.289158  0.177918
        2022-03-21 13:30:00  0.298127  0.290705  0.294858  0.291086  0.148927
...                               ...       ...       ...       ...       ...
        2025-02-13 15:30:00  0.869258  0.869372  0.864440  0.866508  0.074582
        2025-02-14 09:30:00  0.890516  0.865841  0.888716  0.866434  0.197734
        2025-02-14 10:30:00  0.887283  0.890443  0.883759  0.882453  0.086259
        2025-02-14 11:30:00  0.886318  0.887173  0.884869  0.886384  0.073202
        2025-02-14 12:30:00  0.887320  0.886318  0.883241  0.883788  0.041692

[5083 rows x 5 columns]

In [43]:
from autogluon.timeseries import TimeSeriesPredictor

predictor = TimeSeriesPredictor(freq='1h', prediction_length=400)
predictor.fit(
    train_data=ts_data,
    presets="bolt_base",
    hyperparameters={
        "Chronos": {"model_path": "amazon/chronos-bolt-base"},
    },
)

Frequency '1h' stored as 'h'
Beginning AutoGluon training...
AutoGluon will save models to '/Users/akramchakrouni/Projects/time-series-forecasting-cluserting/notebooks/AutogluonModels/ag-20250214_185606'
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.11
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.2.0: Fri Dec  6 19:02:41 PST 2024; root:xnu-11215.61.5~2/RELEASE_ARM64_T6030
CPU Count:          11
GPU Count:          0
Memory Avail:       6.24 GB / 18.00 GB (34.7%)
Disk Space Avail:   377.24 GB / 460.43 GB (81.9%)
Setting presets to: bolt_base

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'h',
 'hyperparameters': {'Chronos': {'model_path': 'amazon/chronos-bolt-base'}},
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 400,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every

In [44]:
forecast = predictor.predict(ts_data)
print(forecast.head())


data with frequency 'None' has been resampled to frequency 'h'.
Model not specified in predict, will default to the model with the best validation score: Chronos[amazon__chronos-bolt-base]


                                 mean       0.1       0.2       0.3       0.4  \
item_id timestamp                                                               
AAPL    2025-02-14 13:00:00  0.893889  0.882970  0.888185  0.890869  0.892642   
        2025-02-14 14:00:00  0.892471  0.876608  0.883882  0.887842  0.890387   
        2025-02-14 15:00:00  0.893507  0.874031  0.883028  0.887789  0.890990   
        2025-02-14 16:00:00  0.897408  0.874104  0.884431  0.890120  0.894120   
        2025-02-14 17:00:00  0.900429  0.873940  0.885616  0.892128  0.896698   

                                  0.5       0.6       0.7       0.8       0.9  
item_id timestamp                                                              
AAPL    2025-02-14 13:00:00  0.893889  0.895113  0.896587  0.898271  0.902394  
        2025-02-14 14:00:00  0.892471  0.894437  0.896744  0.899629  0.905584  
        2025-02-14 15:00:00  0.893507  0.896108  0.898883  0.902309  0.909636  
        2025-02-14 16:00:00  0.8

In [45]:
evaluation_results = predictor.evaluate(ts_data)
print(evaluation_results)


data with frequency 'None' has been resampled to frequency 'h'.
Model not specified in predict, will default to the model with the best validation score: Chronos[amazon__chronos-bolt-base]


{'WQL': -0.030171280564278553}


In [46]:
import matplotlib.pyplot as plt

# Suppose forecast has a similar structure to your original ts_data
plt.figure(figsize=(12, 6))
plt.plot(ts_data.query("item_id == 'AAPL'")['timestamp'], ts_data.query("item_id == 'AAPL'")['target'], label="Actual")
plt.plot(forecast.query("item_id == 'AAPL'")['timestamp'], forecast.query("item_id == 'AAPL'")['target'], label="Forecast")
plt.legend()
plt.xlabel("Timestamp")
plt.ylabel("Target")
plt.title("AAPL Forecast vs Actual")
plt.xticks(rotation=45)
plt.show()


KeyError: 'timestamp'

<Figure size 1200x600 with 0 Axes>